In [4]:
import numpy as np

class EnvironmentSetup:
  global used_indices

  # function to generate a zero matrix of size n x m
  def generateMatrix(self, row, col):
      return np.zeros((row, col), dtype=int)

  # function to generate random indices for packages, drop location and obstacles in matrix
  used_indices = set()
  def generateRandomIndices(self, row, col, n):
      np.random.seed(59)
      indices = []
      while n > 0:
          index = (np.random.randint(0, row), np.random.randint(0, col))
          if index not in used_indices:
              indices.append(index)
              used_indices.add(index)
              n -= 1
      return indices

  # function to place the symbols on the indices for package, drop, and obstacles in the warehouse
  def placeSymbols(self, s, indices, warehouse):
      for index in indices:
          warehouse[index[0], index[1]] = s
      return warehouse

  def displayEnvironment(self, warehouse):
    print(warehouse)

In [7]:
import heapq
class Agent:
    def __init__(self):
      self.graph = {}

# This func adds edges in the graph with their cost
    def add_edge(self, u, v, cost=1):
      if u not in self.graph:
        self.graph[u] = []
      self.graph[u].append((cost, v))

# Creates a graph with the neighbour elements of a node
    def create_graph(self, warehouse):
      for row in range(len(warehouse)):
        for col in range(len(warehouse[0])):
          self.get_neighbours(row, col)

# Uniform cost search algorithm to return the path with cost of the path
    def ucs(self, start, goal):
        visited = {start: (0, None)}
        p_queue = []
        heapq.heappush(p_queue, (start, 0, []))
        while p_queue:
          node, cost, path = heapq.heappop(p_queue)
          if node == goal:
            print(visited)
            return path+[node], cost

          for edge_cost, neighbour in self.graph[node]:
            if neighbour not in visited or edge_cost + cost < visited[neighbour][0]:
              visited[neighbour] = (edge_cost + cost, node)
              heapq.heappush(p_queue, (neighbour, edge_cost + cost, path+[node]))

        return None, None

# checks if the move is valid or not (is there any obstacle present on the path)
    def is_valid_move(self, row, col):
      return 0 <= row < self.rows and 0 <= col < self.cols and self.grid[row][col] != 3

# returns the neighbour nodes from the location of the element
    def get_neighbours(self, row, col):
      neighbours = []
      for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < self.rows and 0 <= new_col < self.cols: # either call is_valid_move here to avoid obstacle at the first place only
          neighbours.append((new_row, new_col))
          self.add_edge((row,col), (new_row, new_col))
      return neighbours

    def pick_up_package(self, p_loc, warehouse):
      self.grid = warehouse
      self.rows = len(warehouse)
      self.cols = len(warehouse[0])

      self.grid[p_loc[0]][p_loc[1]] = 0
      print(f"Picked up package at {p_loc}")

    def drop_package(self, d_loc, warehouse):
      self.grid = warehouse
      self.rows = len(warehouse)
      self.cols = len(warehouse[0])

      self.grid[d_loc[0]][d_loc[1]] = 1
      self.total_Reward =+ 10
      print(f"Dropped package at {d_loc}")

      pass

    def move_agent(self, start, warehouse, packageLocations, dropLocations):
      self.grid = warehouse
      self.rows = len(warehouse)
      self.cols = len(warehouse[0])
      self.create_graph(warehouse)
      self.total_cost = 0
      current_location = start

      for package, drop in zip(packageLocations, dropLocations):
        target_location = package
        path, cost = self.ucs(current_location, target_location)

        if path:
          for i, node in enumerate(path[1:]):
            if self.is_valid_move(node[0], node[1]):
              print(f'Moving from {current_location} to {node}')
              current_location = node
            else:
              current_location = path[i-1]
            continue
          self.total_cost += 1
          self.pick_up_package(package, self.grid)

        target_location = drop
        path, cost = self.ucs(current_location, target_location)

        if path:
          for i, node in enumerate(path[1:]):
            if self.is_valid_move(node[0], node[1]):
              print(f'Moving from {current_location} to {node}')
              current_location = node
            else:
              current_location = path[i-1]
            continue
          self.total_cost += 1
          self.drop_package(drop, self.grid)

      return self.total_cost

    def main(self, n, m, num_package, num_obstacle):
      environment = EnvironmentSetup()
      self.n = n
      self.m = m
      self.num_package = num_package
      self.num_obstacle = num_obstacle

      warehouse = environment.generateMatrix(n, m)
      self.start = environment.generateRandomIndices(n, m, 1)
      packageLocations = environment.generateRandomIndices(n, m, num_package)
      dropLocations = environment.generateRandomIndices(n, m, num_package)
      obstacleLocations = environment.generateRandomIndices(n, m, num_obstacle)


      warehouse = environment.placeSymbols(1, packageLocations, warehouse)
      warehouse = environment.placeSymbols(2, dropLocations, warehouse)
      warehouse = environment.placeSymbols(3, obstacleLocations, warehouse)

      environment.displayEnvironment(warehouse)
      self.move_agent(self.start[0], warehouse, packageLocations, dropLocations)

In [8]:
n = 5
m = 6
num_package = 3
num_obstacle = 4

agent = Agent()
agent.main(n,m, num_package, num_obstacle)

[[0 0 0 0 3 0]
 [1 0 3 0 0 0]
 [3 2 2 0 0 0]
 [1 0 2 0 0 1]
 [0 3 0 0 0 0]]
{(1, 4): (0, None), (1, 5): (1, (1, 4)), (1, 3): (1, (1, 4)), (2, 4): (1, (1, 4)), (0, 4): (1, (1, 4)), (0, 5): (2, (0, 4)), (0, 3): (2, (0, 4)), (0, 2): (3, (0, 3)), (0, 1): (4, (0, 2)), (1, 2): (2, (1, 3)), (0, 0): (5, (0, 1)), (1, 1): (3, (1, 2)), (1, 0): (4, (1, 1)), (2, 0): (5, (1, 0)), (2, 1): (4, (1, 1)), (2, 2): (3, (1, 2)), (2, 3): (2, (1, 3)), (2, 5): (2, (1, 5)), (3, 0): (6, (2, 0)), (3, 1): (5, (2, 1)), (3, 2): (4, (2, 2)), (3, 3): (3, (2, 3)), (3, 4): (2, (2, 4)), (3, 5): (3, (2, 5)), (4, 0): (7, (3, 0)), (4, 1): (6, (3, 1)), (4, 2): (5, (3, 2)), (4, 3): (4, (3, 3)), (4, 4): (3, (3, 4))}
Moving from (1, 4) to (1, 5)
Moving from (1, 5) to (2, 5)
Moving from (2, 5) to (3, 5)
Picked up package at (3, 5)
{(3, 5): (0, None), (3, 4): (1, (3, 5)), (4, 5): (1, (3, 5)), (2, 5): (1, (3, 5)), (2, 4): (2, (2, 5)), (1, 5): (2, (2, 5)), (1, 4): (3, (1, 5)), (0, 5): (3, (1, 5)), (0, 4): (4, (0, 5)), (0, 3): (5, (